<a href="https://colab.research.google.com/github/noobcoder36/LSAP_usingColab/blob/main/FNN4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Data and Training and Saving Models

In [ ]:
#EE5140 Course Project: 
# Authors : Tejaram S, Nitin S, Bharati K
# Recreating results from paper : https://ieeexplore.ieee.org/document/8371290 

import numpy as np
from scipy.optimize import linear_sum_assignment
import tensorflow as tf 
from tensorflow import keras

import os 

n    = 4      # No. of Jobs, People 
m    = 45000   # No, of training samples 
test = 5000   # No, of test samples

C = np.random.randint(1,100, size=(m,n,n)) 
X = np.zeros((m,n,n),dtype=int)
for i in range(m):
    row_ind, col_ind = linear_sum_assignment(C[i])
    X[i,row_ind,col_ind] = 1

C_test = np.random.randint(1,100, size=(test,n,n)) 
X_test = np.zeros((test,n,n),dtype=int)

for ii in range(test):
    row_ind, col_ind = linear_sum_assignment(C_test[ii])
    X_test[ii,row_ind,col_ind] = 1

# Building C(m) and X(m)-->dataset m--training sample index
# Need to check if training and test data need to be separated before saving
# Check https://www.tensorflow.org/tutorials/load_data/numpy for model creation

print('Cost matrix [i/p]:\n',C)
print('Decision Matrix [o/p]:\n',X)
print('-'*80)
X_1 = X[:,:,0]
X_2 = X[:,:,1]
X_3 = X[:,:,2]
X_4 = X[:,:,3]

X_t1 = X_test[:,:,0]
X_t2 = X_test[:,:,1]
X_t3 = X_test[:,:,2]
X_t4 = X_test[:,:,3]

BATCH_SIZE = 1024
SHUFFLE_BUFFER_SIZE = 256

Cost matrix [i/p]:
 [[[18 59 60 89]
  [36 12 82 36]
  [37 25 68 88]
  [94 20 82 23]]

 [[68 21 39 43]
  [73 34 69 76]
  [ 2 78 13 18]
  [21 62 56 65]]

 [[72 60 26 50]
  [34 57 84 96]
  [65 17 74 98]
  [41 94 41 45]]

 ...

 [[84 49  7 53]
  [29  5  8 75]
  [68 10 16 21]
  [12 99 26 86]]

 [[75 15 91 21]
  [91 90 77 86]
  [ 2 15 14 49]
  [24 50  1 18]]

 [[94 76 42 13]
  [80 71 29 39]
  [32  8 91 76]
  [73 55 13 64]]]
Decision Matrix [o/p]:
 [[[1 0 0 0]
  [0 1 0 0]
  [0 0 1 0]
  [0 0 0 1]]

 [[0 0 0 1]
  [0 1 0 0]
  [0 0 1 0]
  [1 0 0 0]]

 [[0 0 1 0]
  [1 0 0 0]
  [0 1 0 0]
  [0 0 0 1]]

 ...

 [[0 0 1 0]
  [0 1 0 0]
  [0 0 0 1]
  [1 0 0 0]]

 [[0 1 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [0 0 1 0]]

 [[0 0 0 1]
  [1 0 0 0]
  [0 1 0 0]
  [0 0 1 0]]]
--------------------------------------------------------------------------------


In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "LSAP/FNN/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # # create a test file to make sure it shows up in the right place
  # !touch 'new_file_in_working_directory.txt'
  # print('\nYour working directory was changed to ' + root_dir + project_folder + \
  #       "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

Mounted at /content/drive


In [ ]:
# Saving the dataset generated for future use. Saving to My Drive/LSAP/FNN. Can be saved at /LSAP to use it for both FNN and CNN (n=4)
np.savez('data_f.npz',C=C,C_test=C_test,X_t1=X_t1,X_t2=X_t2,X_t3=X_t3,X_t4=X_t4,X_1=X_1,X_2=X_2,X_3=X_3,X_4=X_4,X_test=X_test)


In [ ]:
# Saving the dataset generated for future use. Saving to My Drive/LSAP
#np.savez('/content/drive/My Drive/LSAP/data_f.npz',C=C,C_test=C_test,X_t1=X_t1,X_t2=X_t2,X_t3=X_t3,X_t4=X_t4,X_1=X_1,X_2=X_2,X_3=X_3,X_4=X_4,X_test=X_test)


In [ ]:
# Reload Saved Data Set inorder to avoid regenerating the dataset
# from google.colab import files
# uploaded = files.upload()

with np.load('data_f.npz') as data:
  C = data['C']
  C_test = data['C_test']
  X_t1=data['X_t1']
  X_t2=data['X_t2']
  X_t3=data['X_t3']
  X_t4=data['X_t4']
  X_1=data['X_1']
  X_2=data['X_2']
  X_3=data['X_3']
  X_4=data['X_4']

In [ ]:
train_dataset1 = tf.data.Dataset.from_tensor_slices((C/100, X_1))
test_dataset1 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t1))

train_dataset1 = train_dataset1.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset1 = test_dataset1.batch(BATCH_SIZE)

# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if(logs.get('loss')<0.01):
#       print("\nReached 60% accuracy so cancelling training!")
#       self.model.stop_training = True

# callbacks = myCallback()
#-------------------------------------------------------------------------
model1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(n, n)),
    tf.keras.layers.Dense(32,  activation = 'sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
    tf.keras.layers.Dense(64,  activation = 'sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
    tf.keras.layers.Dense(256, activation = 'sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
    tf.keras.layers.Dense(4,   activation = 'softmax',kernel_regularizer=keras.regularizers.l2(l=0.0001))
])
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model1.summary()

model1.compile(optimizer=opt, loss='categorical_crossentropy',  metrics=['accuracy'])

model1.fit(train_dataset1, epochs=500)

model1.evaluate(test_dataset1)

## Added on Dec 1 9.50 PM
!mkdir -p -v saved_model
model1.save('saved_model/model1')


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 1028      
Total params: 20,324
Trainable params: 20,324
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
44/44 [==============================] - 0s 7ms/step - loss: 1.4243 - accuracy: 0.2523
Epoch 2/500
44/44 [

In [ ]:
 train_dataset2 = tf.data.Dataset.from_tensor_slices((C/100, X_2))
 test_dataset2 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t2))

 train_dataset2 = train_dataset2.batch(BATCH_SIZE).shuffle(SHUFFLE_BUFFER_SIZE)
 test_dataset2 = test_dataset2.batch(BATCH_SIZE)

 model2 = tf.keras.Sequential([
     tf.keras.layers.Flatten(input_shape=(n, n)),
     tf.keras.layers.Dense(32, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(64, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(256, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(4,activation='softmax',kernel_regularizer=keras.regularizers.l2(l=0.0001))
 ])

 model2.compile(optimizer='adam',  loss='categorical_crossentropy',  metrics=['accuracy'])

 model2.fit(train_dataset2, epochs=500)

 model2.evaluate(test_dataset2)

 ## Added on Dec 1 9.50 PM
# !mkdir -p saved_model2
model2.save('saved_model/model2')

Epoch 1/500
44/44 [==============================] - 0s 6ms/step - loss: 1.4460 - accuracy: 0.2533
Epoch 2/500
44/44 [==============================] - 0s 6ms/step - loss: 1.4011 - accuracy: 0.2799
Epoch 3/500
44/44 [==============================] - 0s 7ms/step - loss: 1.3973 - accuracy: 0.3028
Epoch 4/500
44/44 [==============================] - 0s 6ms/step - loss: 1.3884 - accuracy: 0.3425
Epoch 5/500
44/44 [==============================] - 0s 6ms/step - loss: 1.3627 - accuracy: 0.4370
Epoch 6/500
44/44 [==============================] - 0s 6ms/step - loss: 1.2807 - accuracy: 0.5783
Epoch 7/500
44/44 [==============================] - 0s 6ms/step - loss: 1.1133 - accuracy: 0.6072
Epoch 8/500
44/44 [==============================] - 0s 6ms/step - loss: 0.9430 - accuracy: 0.6553
Epoch 9/500
44/44 [==============================] - 0s 6ms/step - loss: 0.8286 - accuracy: 0.6959
Epoch 10/500
44/44 [==============================] - 0s 6ms/step - loss: 0.7387 - accuracy: 0.7316
Epoch 11/

In [ ]:
 train_dataset3 = tf.data.Dataset.from_tensor_slices((C/100, X_3))
 test_dataset3 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t3))


 train_dataset3 = train_dataset3.batch(BATCH_SIZE)
 test_dataset3 = test_dataset3.batch(BATCH_SIZE)

 model3 = tf.keras.Sequential([
     tf.keras.layers.Flatten(input_shape=(n, n)),
     tf.keras.layers.Dense(32, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(64, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(256, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(4,activation='softmax',kernel_regularizer=keras.regularizers.l2(l=0.0001))
 ])

 model3.compile(optimizer='adam',   loss='categorical_crossentropy',    metrics=['accuracy'])

 model3.fit(train_dataset3, epochs=500)

 model3.evaluate(test_dataset3)

 model3.save('saved_model/model3')

Epoch 1/500
44/44 [==============================] - 0s 7ms/step - loss: 1.4099 - accuracy: 0.2548
Epoch 2/500
44/44 [==============================] - 0s 7ms/step - loss: 1.3996 - accuracy: 0.2601
Epoch 3/500
44/44 [==============================] - 0s 7ms/step - loss: 1.3921 - accuracy: 0.3066
Epoch 4/500
44/44 [==============================] - 0s 7ms/step - loss: 1.3689 - accuracy: 0.4457
Epoch 5/500
44/44 [==============================] - 0s 8ms/step - loss: 1.2882 - accuracy: 0.5832
Epoch 6/500
44/44 [==============================] - 0s 7ms/step - loss: 1.1083 - accuracy: 0.6462
Epoch 7/500
44/44 [==============================] - 0s 7ms/step - loss: 0.8994 - accuracy: 0.7049
Epoch 8/500
44/44 [==============================] - 0s 7ms/step - loss: 0.7350 - accuracy: 0.7416
Epoch 9/500
44/44 [==============================] - 0s 7ms/step - loss: 0.6582 - accuracy: 0.7562
Epoch 10/500
44/44 [==============================] - 0s 7ms/step - loss: 0.6318 - accuracy: 0.7599
Epoch 11/

In [ ]:
 train_dataset4 = tf.data.Dataset.from_tensor_slices((C/100, X_4))
 test_dataset4 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t4))


 train_dataset4 = train_dataset4.batch(BATCH_SIZE)
 test_dataset4 = test_dataset4.batch(BATCH_SIZE)

 model4 = tf.keras.Sequential([
     tf.keras.layers.Flatten(input_shape=(n, n)),
     tf.keras.layers.Dense(32, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(64, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(256, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(l=0.0001)),
     tf.keras.layers.Dense(4,activation='softmax',kernel_regularizer=keras.regularizers.l2(l=0.0001))
 ])

 model4.compile(optimizer='adam',   loss='categorical_crossentropy',   metrics=['accuracy'])

 model4.fit(train_dataset4, epochs=500)

 model4.evaluate(test_dataset4)

 model4.save('saved_model/model4')

Epoch 1/500
44/44 [==============================] - 0s 7ms/step - loss: 1.4320 - accuracy: 0.2566
Epoch 2/500
44/44 [==============================] - 0s 7ms/step - loss: 1.4015 - accuracy: 0.2614
Epoch 3/500
44/44 [==============================] - 0s 7ms/step - loss: 1.3969 - accuracy: 0.2757
Epoch 4/500
44/44 [==============================] - 0s 7ms/step - loss: 1.3862 - accuracy: 0.3500
Epoch 5/500
44/44 [==============================] - 0s 7ms/step - loss: 1.3528 - accuracy: 0.4538
Epoch 6/500
44/44 [==============================] - 0s 8ms/step - loss: 1.2584 - accuracy: 0.5284
Epoch 7/500
44/44 [==============================] - 0s 7ms/step - loss: 1.1223 - accuracy: 0.5881
Epoch 8/500
44/44 [==============================] - 0s 8ms/step - loss: 0.9940 - accuracy: 0.6485
Epoch 9/500
44/44 [==============================] - 0s 7ms/step - loss: 0.8777 - accuracy: 0.6992
Epoch 10/500
44/44 [==============================] - 0s 7ms/step - loss: 0.7808 - accuracy: 0.7268
Epoch 11/

# Reload Saved Models and Test Accuracy for 5000 samples [FNN] [To be completed]

In [ ]:
# import numpy as np
# from scipy.optimize import linear_sum_assignment
# import tensorflow as tf 
# from tensorflow import keras

In [ ]:
# # Reload Saved Data Set inorder to avoid retraining model
# from google.colab import files
# uploaded = files.upload()

# with np.load('data_f.npz') as data:
#   C = data['C']
#   C_test = data['C_test']
#   X_t1=data['X_t1']
#   X_t2=data['X_t2']
#   X_t3=data['X_t3']
#   X_t4=data['X_t4']
#   X_1=data['X_1']
#   X_2=data['X_2']
#   X_3=data['X_3']
#   X_4=data['X_4']

In [ ]:
# # Saved Changes on Dec 1 
# # Things to clarified; Training multpile models on the same graph and session and loading in the same graph and session
# # Will this lead to wrong results
# # To be done
# # Interpret ouputs [[g1 g2 g3 g4]] 5000*4*4 and Get Xj and Apply Greedy Collision Rule{if Cij1 < Cij2: assign job i to person j1 } and Get the Inferred Final Ouput
# # Now get accuracy of the results with corresponding Hungarain Algo's Output
# # Saving Model will find its use when implemented on PC than on colab. As colab runs as if everything is on same script.

# #1
# n_model1 = tf.keras.models.load_model('saved_model/model1')

# # Check its architecture
# n_model1.summary()
# g1= n_model1.predict(test_dataset1)

# #2
# n_model2 = tf.keras.models.load_model('saved_model/model2')

# # Check its architecture
# n_model2.summary()
# g2= n_model2.predict(test_dataset2)

# #3
# n_model3 = tf.keras.models.load_model('saved_model/model3')

# # Check its architecture
# n_model3.summary()
# g3= n_model3.predict(test_dataset3)


# #4
# n_model4 = tf.keras.models.load_model('saved_model/model4')
# # Check its architecture
# n_model4.summary()
# g4 = n_model4.predict(test_dataset4)
# g41= n_model4.evaluate(test_dataset4)

# print('Predict:{}'.format(g4[0]))
# print('Evaluate:{}'.format(g41[0]))
# print(g41)

# print(g1.shape)
# print(g2.shape)
# print(g3.shape)
# print(g1[0])
# print(g2)

In [ ]:
# # X_cap = np.zeros((test,n,n),dtype=int)
# # job = np.zeros((test,n),dtype=int)
# p= [0, 1, 2, 3] # Persons {0..j}

# count = np.zeros((test,1),dtype=int)

# for s in range(test):
#     job[s,p[0]] = np.argmax(g1[s])
#     job[s,p[1]] = np.argmax(g2[s])
#     job[s,p[2]] = np.argmax(g3[s])
#     job[s,p[3]] = np.argmax(g4[s])

# for s in range(test):
#     if (job[s,p[0]]==job[s,p[1]]):
#         count[s]+=1
#     if (job[s,p[0]]==job[s,p[2]]):
#         count[s]+=1
#     if (job[s,p[0]]==job[s,p[3]]):
#         count[s]+=1
#     if (job[s,p[1]]==job[s,p[2]]):
#         count[s]+=1
#     if (job[s,p[1]]==job[s,p[3]]):
#         count[s]+=1
#     if (job[s,p[2]]==job[s,p[3]]):
#         count[s]+=1

# k=0
# sum1=0
# sum2=0
# sum3=0
# sum4=0
# sum5=0
# sum6=0

# for s in range(test):
#   if(count[s]==0):
#     k+=1
#   if(count[s]==1):
#     sum1+=1
#   if(count[s]==2):
#     sum2+=1
#   if(count[s]==3):
#     sum3+=1
#   if(count[s]==4):
#     sum4+=1
#   if(count[s]==5):
#     sum5+=1
#   if(count[s]==6):
#     sum6+=1

# print('Collsion Count--> 0: {}| 1: {} | 2: {} | 3: {} | 4: {} | 5: {} | 6: {}'.format(k,sum1,sum2,sum3,sum4,sum5,sum6))

# for s in range(test):
#   X_cap[s,job[s,p[0]],p[0]] = 1
#   X_cap[s,job[s,p[1]],p[1]] = 1
#   X_cap[s,job[s,p[2]],p[2]] = 1
#   X_cap[s,job[s,p[3]],p[3]] = 1 

# print('Before Greedy:\n {}'.format(X_cap[0]))


In [ ]:
# r = np.zeros((test,1),dtype=int)
# for s in range(test):
#   if count[s]==1:

#     for ind1 in range(n):
#       r_s = X_cap[s,ind1,:] == np.zeros((n,),dtype=int)
#       if (r_s.all()):
#         r[s]=ind1
#   else: 
#     r[s] = -1

# print('Before:\n {}'.format(X_cap[0:10]))
# print('Rows----------')
# print(r[0:10])

In [ ]:
# print(count[0:10])
# print('Zero row {}'.format(r[0]))
# print('Before Greedy:\n {}'.format(X_cap[0]))
# for s in range(test):
#   if count[s]==1:
#         #Greedy Collison Rule  
#         if (job[s,p[0]]==job[s,p[1]]):
#             if (C_test[s,job[s,p[0]],p[0]] + C_test[s,r[s],p[1]] < C_test[s,job[s,p[1]],p[1]] + C_test[s,r[s],p[0]]):
#                 X_cap[s,job[s,p[0]],p[0]] = 1
#                 X_cap[s,job[s,p[1]],p[1]] = 0
#                 X_cap[s,r[s],p[1]]        = 1
#                 X_cap[s,r[s],p[0]]        = 0
#             else:
#                 X_cap[s,job[s,p[1]],p[1]] = 1
#                 X_cap[s,job[s,p[0]],p[0]] = 0
#                 X_cap[s,r[s],p[0]]        = 1
#                 X_cap[s,r[s],p[1]]        = 0

#         elif (job[s,p[0]]==job[s,p[2]]):
#             if (C_test[s,job[s,p[0]],p[0]] + C_test[s,r[s],p[2]]  < C_test[s,job[s,p[2]],p[2]]  + C_test[s,r[s],p[0]]):
#                 X_cap[s,job[s,p[0]],p[0]] = 1
#                 X_cap[s,job[s,p[2]],p[2]] = 0
#                 X_cap[s,r[s],p[2]]        = 1
#                 X_cap[s,r[s],p[0]]        = 0
#             else:
#                 X_cap[s,job[s,p[2]],p[2]] = 1
#                 X_cap[s,job[s,p[0]],p[0]] = 0
#                 X_cap[s,r[s],p[0]]        = 1
#                 X_cap[s,r[s],p[2]]        = 0

#         elif (job[s,p[0]]==job[s,p[3]]):
#             if (C_test[s,job[s,p[0]],p[0]] + C_test[s,r[s],p[3]]  < C_test[s,job[s,p[3]],p[3]] + C_test[s,r[s],p[0]]):
#                 X_cap[s,job[s,p[0]],p[0]] = 1
#                 X_cap[s,job[s,p[3]],p[3]] = 0
#                 X_cap[s,r[s],p[3]]        = 1
#                 X_cap[s,r[s],p[0]]        = 0 
#             else:
#                 X_cap[s,job[s,p[3]],p[3]] = 1
#                 X_cap[s,job[s,p[0]],p[0]] = 0
#                 X_cap[s,r[s],p[0]]        = 1
#                 X_cap[s,r[s],p[3]]        = 0

#         elif (job[s,p[1]]==job[s,p[2]]):
#             if (C_test[s,job[s,p[1]],p[1]]   + C_test[s,r[s],p[2]]  < C_test[s,job[s,p[2]],p[2]] + C_test[s,r[s],p[1]]):
#                  X_cap[s,job[s,p[1]],p[1]] = 1
#                  X_cap[s,job[s,p[2]],p[2]] = 0
#                  X_cap[s,r[s],p[2]]        = 1
#                  X_cap[s,r[s],p[1]]        = 0 
#             else:
#                 X_cap[s,job[s,p[2]],p[2]] = 1
#                 X_cap[s,job[s,p[1]],p[1]] = 0
#                 X_cap[s,r[s],p[1]]        = 1
#                 X_cap[s,r[s],p[2]]        = 0

#         elif (job[s,p[1]]==job[s,p[3]]):
#             if (C_test[s,job[s,p[1]],p[1]]  +  C_test[s,r[s],p[3]]  <  C_test[s,job[s,p[3]],p[3]]  +  C_test[s,r[s],p[1]]):
#                 X_cap[s,job[s,p[1]],p[1]] = 1
#                 X_cap[s,job[s,p[3]],p[3]] = 0
#                 X_cap[s,r[s],p[3]]        = 1
#                 X_cap[s,r[s],p[1]]        = 0 
#             else:
#                 X_cap[s,job[s,p[3]],p[3]] = 1
#                 X_cap[s,job[s,p[1]],p[1]] = 0
#                 X_cap[s,r[s],p[1]]        = 1
#                 X_cap[s,r[s],p[3]]        = 0

#         elif (job[s,p[2]]==job[s,p[3]]):
#             if (C_test[s,job[s,p[2]],p[2]]   +  C_test[s,r[s],p[3]]  < C_test[s,job[s,p[3]],p[3]]  +   C_test[s,r[s],p[2]]):
#                  X_cap[s,job[s,p[2]],p[2]] = 1
#                  X_cap[s,job[s,p[3]],p[3]] = 0
#                  X_cap[s,r[s],p[3]]        = 1
#                  X_cap[s,r[s],p[2]]        = 0
#             else:
#                 X_cap[s,job[s,p[3]],p[3]] = 1
#                 X_cap[s,job[s,p[2]],p[2]] = 0
#                 X_cap[s,r[s],p[2]]        = 1
#                 X_cap[s,r[s],p[3]]        = 0


# print('After Greedy:\n {}'.format(X_cap[0]))

# # Testing Accuracy 
# error = 0
# for s in range(test):
#   comparison = X_cap[s]  == X_test[s]
#   ind = comparison.all()
#   if not ind:
#     error+=1

# accuracy = (test-error)/test

# print('Error:{}'.format(error))
# print('Accuracy:{}'.format(accuracy))

# print('Jobs for each person:{}'.format(job[0]))

# print('Original Sol:\n{}'.format(X_test[0]))
# print('Cost matrix 0th:\n{}'.format(C_test[0]))


In [ ]:
# # Added on Dec 13 1:07AM
# # Reducing One Collision Event, by finding index for intersection of row and column with all zeros 
# for s in range(1):
#   if count[s]==1:
#     print('Before:\n {}'.format(X_cap[s]))
#     print('Rows----------')
#     for ind1 in range(n):
#       r = X_cap[s,ind1,:]== np.zeros((n,),dtype=int)
#       print(r)
#       if (r.all()):
#         row=ind1
#     print('Columns----------')
#     for ind2 in range(n):
#       c = X_cap[s,:,ind2]== np.zeros((n,),dtype=int)
#       print(c)
#       if (c.all()):
#         col=ind2
#     X_cap[s,row,col] = 1
#     print(row)
#     print(col)
#     print('After:\n {}'.format(X_cap[s]))   

# error1 = 0
# for s in range(test):
#   comparison1 = X_cap[s]  == X_test[s]
#   ind = comparison.all()
#   if not ind:
#     error1+=1

# print('Error:{}'.format(error1))


# Computing Time for Hungarian Algorithm



> First Approach Involving Average



In [ ]:
# # Saved Changes on Dec 1 ; 9.43PM
# import timeit

    
# setup_code = '''
# from scipy.optimize import linear_sum_assignment
# import numpy as np
# n    = 4      # No. of Jobs, People 
# test = 5000   # No, of test samples
# C_test = np.random.randint(1,100, size=(test,n,n)) 
# X_test = np.zeros((test,n,n),dtype=int)
# '''
# statement = """
# for ii in range(test):
#     row_ind, col_ind = linear_sum_assignment(C_test[ii])
#     X_test[ii,row_ind,col_ind] = 1
# """
# num = 100
# time = timeit.timeit(setup = setup_code, stmt = statement, number = num)
# avg = time/num
# print("Execution time for {} iterations is: {}".format(num,time))
# print("Avg Time:{}".format(avg))



> Second One Using Subtraction of time between epoch



In [ ]:
# import time

# # starting time
# start = time.time()

# from scipy.optimize import linear_sum_assignment
# import numpy as np
# n    = 4      # No. of Jobs, People 
# test = 5000   # No, of test samples

# C_test = np.random.randint(1,100, size=(test,n,n)) 
# X_test = np.zeros((test,n,n),dtype=int)
# for ii in range(test):
#     row_ind, col_ind = linear_sum_assignment(C_test[ii])
#     X_test[ii,row_ind,col_ind] = 1
# # program body ends

# # end time
# end = time.time()

# # total time taken
# print(f"Runtime of the program is {end - start}")